In [ ]:
from B1Py.calibration import KalibrExtractExtrinsics
from B1Py.calibration import Vicon2GtExtractParams
from B1Py.calibration import ExtrinsicCalibrationManager
import numpy as np

In [ ]:
import rclpy
rclpy.init()

In [ ]:
import os
camera_intrinsics_dir = '/home/rstaion/Data-HDD/datasets/calibration/B1/kalibr/static/Results/all'
camera_extrinsics_dir = '/home/rstaion/Data-HDD/datasets/calibration/B1/simplehandeye/results'
vicon2gt_result_file = '/home/rstaion/Data-HDD/datasets/calibration/B1/vicon2gt/final/result/vicon2gt-seq1_vicon2gt_info.txt'

In [ ]:
import yaml
def KalibrExtractSensorToFrameNameMap(param_file):
    '''
    Given the path to a kalibr calibration yaml file, returns a dictionary mapping sensor names in the yaml file 
    to the corresponding frame ids as published by realsense_ros ROS2 package. This has been done by inspecting the
    rostopic field in the yaml file.
    
    @ param param_file: path to a kalibr calibration yaml file
    @ return: dictionary mapping sensor names in the yaml file to the corresponding frame ids as published by realsense_ros ROS2 package.
    '''
    with open(param_file, 'r') as f:
        data = yaml.safe_load(f)

    sensor_name_to_frame_id = {}
    for sensor in data.keys():  
        tmp = data[sensor]['rostopic'].split('/')
        frame_name = tmp[2]+'_'+tmp[3]+'_optical_frame'
        sensor_name_to_frame_id[sensor] = frame_name
    return sensor_name_to_frame_id

def KalibrExtractFrameExtrinsics(camera_intrinsics_dir):
    '''
    Given the directory containing the result of kalibr calibration procedure, returns 
    a dictionary of 4x4 numpy arrays representing the extrinsics between the parent and child frames
    with parent and child frames having similar TF names as the ones published by realsense_ros ROS2 package. 

    @ param camera_intrinsics_dir: directory containing a collection yaml files from kalibr calibration procedure
    @ return: dictionary of 4x4 numpy arrays representing the extrinsics between the parent and child frames info['parent_frame_id:child_frame_id'] = pose
    '''
    intrinsics_files = os.listdir(camera_intrinsics_dir)
    info = {} # info will hold the extrinsics poses between parent frame ids and child frame ids as info['parent_frame_id:child_frame_id'] = pose
    for file in intrinsics_files:
        param_file = os.path.join(camera_intrinsics_dir,file)
        ext_params = KalibrExtractExtrinsics(param_file)
        if len(ext_params) == 0:
            continue

        map = KalibrExtractSensorToFrameNameMap(param_file)
        for pose in ext_params:
            parent_sensor_name = pose.split('_T_')[0]
            child_sensor_name = pose.split('_T_')[-1]
            if child_sensor_name == 'imu':
                child_frame_id = 'b1_imu_link'
            else:
                child_frame_id = map[child_sensor_name]

            parent_frame_id = map[parent_sensor_name]
            info[f'{parent_frame_id}:{child_frame_id}'] = ext_params[pose]
    return info

In [ ]:
def SimpleHandEyeExtractFrameExtrinsics(camera_extrinsics_dir):
    '''
    Given the directory containing the result of simplehandeye calibration procedure, returns
    a dictionary of 4x4 numpy arrays representing the extrinsics between the parent and child frames
    as identified by the simplehandeye calibration procedure. The parent frame name is the same as the one 
    published by the vicon_bridge2 ROS2 package while the child frame name is the same as the one published by  
    realsense_ros ROS2 package.

    @ param camera_extrinsics_dir: directory containing a collection yaml files from simplehandeye calibration procedure
    @ return: dictionary of 4x4 numpy arrays representing the extrinsics between the parent and child frames info['parent_frame_id:child_frame_id'] = pose
    '''
    extrinsics_files = os.listdir(camera_extrinsics_dir)
    info = {}
    for file in extrinsics_files:
        param_file = os.path.join(camera_extrinsics_dir, file)
        with open(param_file, 'r') as f:
            data = yaml.safe_load(f)

        parent_frame_id = data['parent_frame']
        for key in data.keys():
            if key == 'parent_frame':
                continue
            child_frame_id = key
            info[f'{parent_frame_id}:{child_frame_id}'] = np.array(data[key])
    return info

In [ ]:
info1 = KalibrExtractFrameExtrinsics(camera_intrinsics_dir)
info2 = SimpleHandEyeExtractFrameExtrinsics(camera_extrinsics_dir)
imu_T_vicon = Vicon2GtExtractParams(vicon2gt_result_file)

In [ ]:
manager = ExtrinsicCalibrationManager()
# Add results from kalibr calibration procedure 
for key in info1:
    parent = key.split(':')[0]
    child = key.split(':')[1]
    manager.add(parent, child, info1[key][0:3,0:3], info1[key][0:3, -1])
# Add results from simplehandeye calibration procedure
for key in info2:
    parent = key.split(':')[0]
    child = key.split(':')[1]
    manager.add(parent, child, info2[key][0:3,0:3], info2[key][0:3, -1])
# Add results from vicon2gt calibration procedure
manager.add('b1_imu_link', 'vicon/B1_BODY/B1_BODY', imu_T_vicon[0:3,0:3], imu_T_vicon[0:3, -1])

In [ ]:
all_tfs = manager.get_all('b1_imu_link')
all_tfs.keys()

In [ ]:
ros_T_optic = np.array([[0, 0, 1, 0],
                        [-1,0, 0, 0],
                        [0, -1, 0, 0],
                        [0, 0, 0, 1]]).astype(np.float64)
keys = [key for key in manager.name_to_id.keys()]
for key in keys:
    if key.endswith('optical_frame'):
        ros_frame = key.split('_optical_frame')[0]+'_frame'
        if key.endswith('infra1_optical_frame'):
            body_frame = key.split('_infra1_optical_frame')[0]+'_link'
        elif key.endswith('infra2_optical_frame'):
            body_frame = key.split('_infra2_optical_frame')[0]+'_link'
        elif key.endswith('color_optical_frame'):
            body_frame = key.split('_color_optical_frame')[0]+'_link'
        elif key.endswith('depth_optical_frame'):
            body_frame = key.split('_depth_optical_frame')[0]+'_link'
        elif key.endswith('imu_optical_frame'):
            body_frame = key.split('_imu_optical_frame')[0]+'_link'
        
        if key.endswith('depth_optical_frame') or key.endswith('infra1_optical_frame'):
            manager.add(body_frame, ros_frame, np.eye(3), np.zeros(3))
        manager.add(ros_frame, key, ros_T_optic[0:3,0:3], ros_T_optic[0:3, -1])

In [ ]:
all_tfs = manager.get_all('b1_d455_cam_link')
for frame in all_tfs:
    print(frame)

In [ ]:
from rclpy.node import Node
import tf2_ros
from geometry_msgs.msg import TransformStamped
from scipy.spatial.transform import Rotation as R
import numpy as np  
from B1Py.robot.interface.ros2 import ROS2ExecutorManager

class RealsenseExtrinsicsBroadcaster(Node):
    def __init__(self, camera_name,  oir1_T_oir2, oir1_T_ocolor, oir1_T_imu=None):
        super().__init__(f'{camera_name}_tf2_broadcaster')

        self.body_frame_name = f'{camera_name}_link'
        self.ir1_frame_name = f'{camera_name}_ir1_frame'
        self.ir2_frame_name = f'{camera_name}_ir2_frame'
        self.color_frame_name = f'{camera_name}_color_frame'
        if oir1_T_imu is not None:
            self.imu_frame_name = f'{camera_name}_imu_frame'
        self.ir1_optical_frame_name = f'{camera_name}_ir1_optical_frame'
        self.ir2_optical_frame_name = f'{camera_name}_ir2_optical_frame'
        self.color_optical_frame_name = f'{camera_name}_color_optical_frame'

        self.oir1_T_imu = oir1_T_imu
        self.oir1_T_oir2 = oir1_T_oir2
        self.oir1_T_ocolor = oir1_T_ocolor

        self.ros_T_optic = np.array([[0, 0, 1, 0],
                                [-1,0, 0, 0],
                                [0, -1, 0, 0],
                                [0, 0, 0, 1]]).astype(np.float64)

        # self.broadcaster = tf2_ros.TransformBroadcaster(self)
        self.broadcaster = tf2_ros.StaticTransformBroadcaster(self)
        self.timer = self.create_timer(1, self.timer_callback)

    def timer_callback(self):
        body_T_ir1 = np.eye(4)
        body_T_ir2 = self.ros_T_optic @ self.oir1_T_oir2 @ np.linalg.inv(self.ros_T_optic)
        body_T_imu = self.ros_T_optic @ self.oir1_T_imu

        tf1 = self.makeTransformStamped(self.body_frame_name, self.ir1_frame_name, body_T_ir1)
        tf2 = self.makeTransformStamped(self.body_frame_name, self.ir2_frame_name, body_T_ir2)
        if self.oir1_T_imu is not None:
            tf3 = self.makeTransformStamped(self.body_frame_name, self.imu_frame_name, body_T_imu)
        tf4 = self.makeTransformStamped(self.ir1_frame_name, self.ir1_optical_frame_name, self.ros_T_optic)
        tf5 = self.makeTransformStamped(self.ir2_frame_name, self.ir2_optical_frame_name, self.ros_T_optic)
        self.broadcaster = tf2_ros.StaticTransformBroadcaster(self)
        self.broadcaster.sendTransform(tf1)
        self.broadcaster.sendTransform(tf2)
        if self.oir1_T_imu is not None:
            self.broadcaster.sendTransform(tf3)
        self.broadcaster.sendTransform(tf4)
        self.broadcaster.sendTransform(tf5)

    def makeTransformStamped(self, parent_frame, child_frame, T):
        rotation = R.from_matrix(T[:3, :3]).as_quat()
        translation = T[:3, 3]
        static_transform_stamped = TransformStamped()
        static_transform_stamped.header.stamp = self.get_clock().now().to_msg()
        static_transform_stamped.header.frame_id = parent_frame
        static_transform_stamped.child_frame_id = child_frame
        static_transform_stamped.transform.translation.x = translation[0]
        static_transform_stamped.transform.translation.y = translation[1]
        static_transform_stamped.transform.translation.z = translation[2]
        static_transform_stamped.transform.rotation.x = rotation[0]
        static_transform_stamped.transform.rotation.y = rotation[1]
        static_transform_stamped.transform.rotation.z = rotation[2]
        static_transform_stamped.transform.rotation.w = rotation[3]
        return static_transform_stamped

In [ ]:
class B1ExtrinsicsBroadcaster(Node):

    def __init__(self, robot_name='b1'):
        super().__init__(f'{robot_name}_tf2_broadcaster')

        self.frames = []
        # self.broadcaster = tf2_ros.TransformBroadcaster(self)
        self.broadcaster = tf2_ros.StaticTransformBroadcaster(self)
        self.timer = self.create_timer(0.5, self.timer_callback)
        # self.body_frame_name = f'{robot_name}_base_link'

    def timer_callback(self):
        for frame in self.frames:
            self.broadcaster.sendTransform(frame)

    def registerFrame(self, parent, child, body_T_sensor):
        frame = self.makeTransformStamped(parent, child, body_T_sensor)
        self.frames.append(frame)

    def makeTransformStamped(self, parent_frame, child_frame, T):
        rotation = R.from_matrix(T[:3, :3]).as_quat()
        translation = T[:3, 3]
        static_transform_stamped = TransformStamped()
        static_transform_stamped.header.stamp = self.get_clock().now().to_msg()
        static_transform_stamped.header.frame_id = parent_frame
        static_transform_stamped.child_frame_id = child_frame
        static_transform_stamped.transform.translation.x = translation[0]
        static_transform_stamped.transform.translation.y = translation[1]
        static_transform_stamped.transform.translation.z = translation[2]
        static_transform_stamped.transform.rotation.x = rotation[0]
        static_transform_stamped.transform.rotation.y = rotation[1]
        static_transform_stamped.transform.rotation.z = rotation[2]
        static_transform_stamped.transform.rotation.w = rotation[3]
        return static_transform_stamped

In [ ]:
b1_broadcaster = B1ExtrinsicsBroadcaster()
for key in all_tfs:
    parent = key.split('_wrt_')[-1]
    child = key.split('_wrt_')[0]
    print(parent, child)
    b1_broadcaster.registerFrame(parent, child, all_tfs[key])

In [ ]:
# rs1_broadcaster = RealsenseExtrinsicsBroadcaster('realsense', oir1_T_oir2, oir1_T_ocolor, oir1_T_imu)
# rs2_broadcaster = RealsenseExtrinsicsBroadcaster('realsense2', oir1_T_oir2, oir1_T_ocolor, oir1_T_imu)

# b1_broadcaster = B1ExtrinsicsBroadcaster()

# T1 = np.eye(4)
# T1[0,-1] = 2.
# T2 = np.eye(4)
# T2[0,-1] = -2.
# b1_broadcaster.registerFrame(rs1_broadcaster.body_frame_name, T1)
# b1_broadcaster.registerFrame(rs2_broadcaster.body_frame_name, T2)

In [ ]:
excution_manager = ROS2ExecutorManager()    
excution_manager.add_node(b1_broadcaster)
excution_manager.start()